<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Generating_Text_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import re
import shutil
import tensorflow as tf

DATA_DIR = './data'
CHECKPOINT_DIR = os.path.join(DATA_DIR, 'checkpoints')

In [2]:
def download_and_read(urls):
  texts = []
  for i, url in enumerate(urls):
    p = tf.keras.utils.get_file('ex1-{:d}.txt'.format(i),url,cache_dir='.')
    text = open(p, 'r').read()
    text = text.replace('\ufeff','')
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+',' ', text)
    texts.extend(text)
  return texts

In [3]:
texts = download_and_read([
    'https://www.gutenberg.org/cache/epub/28885/pg28885.txt',
    'https://www.gutenberg.org/files/12/12-0.txt'
])

In [4]:
vocab = sorted(set(texts))
print('vocab size: ',len(vocab))

char2idx = {c:i for i, c in enumerate(vocab)}
idx2char = {i:c for c, i in char2idx.items()}

vocab size:  90


In [5]:
texts_as_ints = np.array([char2idx[c] for c in texts])
data = tf.data.Dataset.from_tensor_slices(texts_as_ints)

seq_length = 100
sequences = data.batch(seq_length+1, drop_remainder=True)

def split_train_labels(sequence):
    input_seq = sequence[0:-1]
    output_seq = sequence[1:]
    return input_seq, output_seq

sequences = sequences.map(split_train_labels)
batch_size = 64
steps_per_epoch = len(texts) // seq_length // batch_size
dataset = sequences.shuffle(10000).batch(batch_size, drop_remainder=True)

In [6]:
class CharGenModel(tf.keras.Model):
  def __init__(self, vocab_size, num_timestamps, embedding_dim, **kwargs):
    super(CharGenModel, self).__init__(**kwargs)
    self.embedding_layer = tf.keras.layers.Embedding(vocab_size,embedding_dim)
    self.rnn_layer = tf.keras.layers.GRU(
        num_timestamps,
        recurrent_initializer = 'glorot_uniform',
        recurrent_activation = 'sigmoid',
        stateful=True,
        return_sequences=True
    )
    self.dense_layer = tf.keras.layers.Dense(vocab_size)

  def call(self, x):
    x = self.embedding_layer(x)
    x = self.rnn_layer(x)
    x = self.dense_layer(x)
    return x

vocab_size = len(vocab)
embedding_dim = 256

In [7]:
def model():
  model = CharGenModel(vocab_size, seq_length, embedding_dim)
  model.build(input_shape=(batch_size,seq_length))
  def loss(labels, predictions):
    return tf.keras.losses.sparse_categorical_crossentropy(
        labels, predictions, from_logits=True
    )
  model.compile(optimizer = tf.keras.optimizers.Adam(), loss=loss)
  return model

In [8]:
def generate_text(model, prefix_string, char2idx, idx2char,
        num_chars_to_generate=100, temperature=1.0):
    input = [char2idx[s] for s in prefix_string]
    input = tf.expand_dims(input, 0)
    text_generated = []
    model.reset_states()
    for i in range(num_chars_to_generate):
        preds = model(input)
        preds = tf.squeeze(preds, 0) / temperature
        # predict char returned by model
        pred_id = tf.random.categorical(preds, num_samples=1)[-1, 0].numpy()
        text_generated.append(idx2char[pred_id])
        # pass the prediction as the next input to the model
        input = tf.expand_dims([pred_id], 0)

    return prefix_string + "".join(text_generated)


In [9]:
with tf.device('GPU:0'):
  num_epochs = 200
  model = model()

  model.fit(
      dataset.repeat(),
      epochs = num_epochs,
      steps_per_epoch = steps_per_epoch
   )
  '''
    print('-'*15 + '\n')
    print('SAVING MODEL CHECKPOINT ', i+1)
    print('-'*15 + '\n')
    
    model.save(os.path.join(CHECKPOINT_DIR, 'model_{:d}'.format(i+1)))
    print('-'*15 + '\n')
    print('GENERATING TEXT...')
    generate_text(model, 'Alice', char2idx, idx2char)
  '''
  model.save(os.path.join(CHECKPOINT_DIR, 'text_generator'))

Epoch 1/200
54/54 [==============================] - 4s 17ms/step - loss: 3.9197
Epoch 2/200
54/54 [==============================] - 1s 15ms/step - loss: 2.8571
Epoch 3/200
54/54 [==============================] - 1s 15ms/step - loss: 2.5209
Epoch 4/200
54/54 [==============================] - 1s 15ms/step - loss: 2.3576
Epoch 5/200
54/54 [==============================] - 1s 15ms/step - loss: 2.2498
Epoch 6/200
54/54 [==============================] - 1s 15ms/step - loss: 2.1538
Epoch 7/200
54/54 [==============================] - 1s 15ms/step - loss: 2.0673
Epoch 8/200
54/54 [==============================] - 1s 15ms/step - loss: 1.9891
Epoch 9/200
54/54 [==============================] - 1s 15ms/step - loss: 1.9353
Epoch 10/200
54/54 [==============================] - 1s 15ms/step - loss: 1.8788
Epoch 11/200
54/54 [==============================] - 1s 15ms/step - loss: 1.8375
Epoch 12/200
54/54 [==============================] - 1s 15ms/step - loss: 1.7950
Epoch 13/200
54/54 [=====

INFO:tensorflow:Assets written to: ./data/checkpoints/text_generator/assets


INFO:tensorflow:Assets written to: ./data/checkpoints/text_generator/assets
